In [26]:
import os
import json
import pandas as pd
import traceback

In [27]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [28]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

False

In [29]:
KEY="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"

In [30]:
llm=ChatGroq(temperature=0, groq_api_key=KEY, model_name="llama3-70b-8192")

In [31]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021A93774610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021A9376B890>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [32]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [33]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [34]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
Do not provide any extra text such as 'Here are the 5 multiple choice questions:'
"""

In [35]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [36]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [37]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}
Check from an expert English Writer of the above quiz:
"""

In [38]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [39]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [40]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [41]:
file_path=r"D:/CODING/GEN AI/mcqgenerator/data.txt"

In [42]:
file_path

'D:/CODING/GEN AI/mcqgenerator/data.txt'

In [43]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [44]:
print(TEXT)

Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once somethin

In [45]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [46]:
NUMBER=5 
SUBJECT="Technology"
TONE="simple"


In [47]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of

In [48]:
response

{'text': 'Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.\n\nSome high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because o

In [49]:
quiz=response.get("quiz")

print(quiz)

{"1": {"mcq": "What is artificial intelligence (AI) in its broadest sense?", "options": {"a": "Intelligence exhibited by humans", "b": "Intelligence exhibited by machines", "c": "Intelligence exhibited by animals", "d": "Intelligence exhibited by robots"}, "correct": "b"}, 
"2": {"mcq": "What is the primary goal of machines with AI?", "options": {"a": "To minimize their chances of achieving defined goals", "b": "To maximize their chances of achieving defined goals", "c": "To ignore their environment", "d": "To interact with humans only"}, "correct": "b"}, 
"3": {"mcq": "Which of the following is an example of a high-profile application of AI?", "options": {"a": "Calculator", "b": "Google Search", "c": "Microsoft Office", "d": "Facebook"}, "correct": "b"}, 
"4": {"mcq": "Why are many AI applications not perceived as AI?", "options": {"a": "Because they are not useful", "b": "Because they are not common", "c": "Because once something becomes useful enough and common enough it's not label

In [50]:
abc=json.loads(quiz)

In [51]:
quiz_table_data = []
for key, value in abc.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [52]:
quiz_table_data

[{'MCQ': 'What is artificial intelligence (AI) in its broadest sense?',
  'Choices': 'a: Intelligence exhibited by humans | b: Intelligence exhibited by machines | c: Intelligence exhibited by animals | d: Intelligence exhibited by robots',
  'Correct': 'b'},
 {'MCQ': 'What is the primary goal of machines with AI?',
  'Choices': 'a: To minimize their chances of achieving defined goals | b: To maximize their chances of achieving defined goals | c: To ignore their environment | d: To interact with humans only',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is an example of a high-profile application of AI?',
  'Choices': 'a: Calculator | b: Google Search | c: Microsoft Office | d: Facebook',
  'Correct': 'b'},
 {'MCQ': 'Why are many AI applications not perceived as AI?',
  'Choices': "a: Because they are not useful | b: Because they are not common | c: Because once something becomes useful enough and common enough it's not labeled AI anymore | d: Because they are not interactive",


In [53]:
quiz=pd.DataFrame(quiz_table_data)

In [54]:
quiz.to_csv("machinelearning.csv",index=False)

In [55]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_19_2024_03_27_36'